In [31]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
from concurrent.futures import ThreadPoolExecutor

In [32]:
def fetch_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"
    except Exception as err:
        return f"An error occurred: {err}"

In [33]:
def get_link(url, date, error_urls):
    try:
        html_content = fetch_url(url + date)
        if not html_content.startswith("HTTP error") and not html_content.startswith("An error"):
            soup = BeautifulSoup(html_content, 'html.parser')
            mydivs = soup.find_all("div", class_="teaser offset")
            with open("oneYearNews8.txt", "a+") as f:
                for div in mydivs:
                    f.write("https://ekantipur.com" + div.find('a')['href'] + "\n")
        else:
            error_urls.append(url + date)
    except Exception as err:
        error_urls.append(url + date)

In [34]:
if __name__ == "__main__":
    url = "https://ekantipur.com/pradesh-7/"
    start_date = date(2023, 1, 1)
    end_date = date(2023, 12, 31)
    delta = timedelta(days=1)
    cus_date = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    error_urls = []

    with ThreadPoolExecutor() as executor:
        executor.map(lambda x: get_link(url, x.strftime("%Y/%m/%d"), error_urls), cus_date)

    with open("error_links.txt", "a+") as f:
        for error_url in error_urls:
            f.write(error_url + "\n")